## Install Libraries


In [1]:
#!pip install git+https://github.com/yfw/starter-code
%pip install ray[tune] bayesian-optimization==1.2.0 scikit-optimize
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 19.6 MB/s eta 0:00:00
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11683 sha256=2228e1e159e87373c5fa9e82e7aa0cf4114cbb71580b71786ceb4b0789194ba6
  Stored in directory: /root/.cache/pip/wheels/14/98/74/190efced439ab92ca551550906290338ba5b64cfc9d0927765
Successfully built bayesian-optimization
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pyp

## Import Libraries


In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split


import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.skopt import SkOptSearch

from ray.tune.search import ConcurrencyLimiter

## Data loaders


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 3
pad_size_3 = 3

fc_count_1 = 1024

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [10]:
def train_cifar(config):
    net = Net()  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = config['momentum']
    weight_decay = 0.001

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                weight_decay=weight_decay,
                momentum=momentum, nesterov=True)    




    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=1
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=1
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = { 
          "learning_rate": tune.uniform(1e-3, 1e-1),
          "momentum": tune.uniform(0.001, 0.9),
          }



In [9]:
# Do not modify any code below!
def main(config):

  algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})

  # current_best_params = [
  #   {
  #       "learning_rate": 0.053306,
  #       "momentum": 0.570568,
  #   },
  #   {
  #       "learning_rate": 0.028210,
  #       "momentum": 0.338465,
  #   },
  #   {
  #       "learning_rate": 0.013342,
  #       "momentum": 0.605203,
  #   }
  # ]

  # algo = SkOptSearch(
  #   metric="accuracy",
  #   mode="max",
  #   points_to_evaluate=current_best_params)

  algo = ConcurrencyLimiter(algo, max_concurrent=1)

  tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(train_cifar),
      resources={"gpu": 1},
    ),
    tune_config=tune.TuneConfig(
      metric="accuracy",
      mode="max",
      num_samples=10,
      search_alg=algo,
    ),
    param_space=config,
    )
  
  results = tuner.fit()

  best_result = results.get_best_result("accuracy", "max")

  print(f"Best trial config: {best_result.config}")
  print(f"Best trial final validation loss: {best_result.metrics['loss']}")
  print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

  test_best_model(best_result)

  return results.get_dataframe("accuracy", "max")


results_df = main(config)


2023-04-14 01:16:55,699	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=1261) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-14_01-16-53/train_cifar_571f383b_1_learning_rate=0.0381,momentum=0.8606_2023-04-14_01-16-57/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 84658577.65it/s]


(train_cifar pid=1261) Extracting /root/ray_results/train_cifar_2023-04-14_01-16-53/train_cifar_571f383b_1_learning_rate=0.0381,momentum=0.8606_2023-04-14_01-16-57/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-14_01-16-53/train_cifar_571f383b_1_learning_rate=0.0381,momentum=0.8606_2023-04-14_01-16-57/data
(train_cifar pid=1261) Files already downloaded and verified


(train_cifar pid=1261) /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(train_cifar pid=1261)   warnings.warn(_create_warning_msg(


...

(train_cifar pid=1261) Extracting /root/ray_results/train_cifar_2023-04-14_01-16-53/train_cifar_57d2028d_10_learning_rate=0.0438,momentum=0.3330_2023-04-14_01-46-36/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-14_01-16-53/train_cifar_57d2028d_10_learning_rate=0.0438,momentum=0.3330_2023-04-14_01-46-36/data
(train_cifar pid=1261) Files already downloaded and verified


2023-04-14 01:49:56,469	INFO tune.py:798 -- Total run time: 1979.06 seconds (1978.69 seconds for the tuning loop).


(train_cifar pid=1261) Finished Training
Best trial config: {'learning_rate': 0.04376255684556946, 'momentum': 0.3329833121584336}
Best trial final validation loss: 0.7950538981492352
Best trial final validation accuracy: 0.7694


100%|██████████| 170498071/170498071 [00:01<00:00, 87240625.40it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.7408


In [11]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum
9,0.691863,0.7699,20.274218,False,123.951019,0.043763,0.332983
7,0.696732,0.7676,18.064580,False,137.223111,0.019001,0.246724
6,0.783483,0.7641,21.025885,False,180.086320,0.083412,0.269871
2,0.770198,0.7628,20.602467,False,205.079264,0.016446,0.224796
8,0.777695,0.7602,17.986834,False,152.805428,0.031120,0.519805
3,0.783662,0.7575,17.602912,False,140.393131,0.006750,0.792941
0,0.768786,0.7533,17.687043,False,166.263623,0.038079,0.860571
5,0.818956,0.7471,17.793905,False,138.655107,0.003038,0.875928
4,NaN,0.1058,25.813243,False,25.813243,0.060510,0.666458
1,NaN,0.0996,23.572686,False,23.572686,0.073467,0.578927


In [12]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum
1,NaN,0.0996,23.572686,False,23.572686,0.073467,0.578927
4,NaN,0.1058,25.813243,False,25.813243,0.060510,0.666458
5,0.818956,0.7471,17.793905,False,138.655107,0.003038,0.875928
0,0.768786,0.7533,17.687043,False,166.263623,0.038079,0.860571
3,0.783662,0.7575,17.602912,False,140.393131,0.006750,0.792941
8,0.777695,0.7602,17.986834,False,152.805428,0.031120,0.519805
2,0.770198,0.7628,20.602467,False,205.079264,0.016446,0.224796
6,0.783483,0.7641,21.025885,False,180.086320,0.083412,0.269871
7,0.696732,0.7676,18.064580,False,137.223111,0.019001,0.246724
9,0.691863,0.7699,20.274218,False,123.951019,0.043763,0.332983
